In [13]:
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn import ensemble

from sklearn.datasets import make_classification

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder

from sklearn.metrics import mean_squared_error #均方误差
from sklearn.metrics import mean_absolute_error #平方绝对误差
from sklearn.metrics import r2_score#R square
from sklearn.metrics import explained_variance_score
from sklearn.metrics import roc_curve

from sklearn import  metrics
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
import matplotlib.pylab as plt
import pandas as pd
import numpy as np
%matplotlib inline

In [16]:
# 弱分类器的数目
n_estimator = 10
# 随机生成分类数据。
X, y = make_classification(n_samples=80000)
# 切分为测试集和训练集，比例0.5
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)
# 将训练集切分为两部分，一部分用于训练GBDT模型，另一部分输入到训练好的GBDT模型生成GBDT特征，然后作为LR的特征。这样分成两部分是为了防止过拟合。
X_train, X_train_lr, y_train, y_train_lr = train_test_split(X_train, y_train, test_size=0.5)
# 调用GBDT分类模型。
grd = ensemble.GradientBoostingClassifier(n_estimators=n_estimator)
# 调用one-hot编码。
grd_enc = OneHotEncoder()
# 调用LR分类模型。
grd_lm = LogisticRegression(max_iter=3000)

#”’使用X_train训练GBDT模型，后面用此模型构造特征”’
grd.fit(X_train, y_train)
# fit one-hot编码器
grd_enc.fit(grd.apply(X_train)[:, :, 0])
#使用训练好的GBDT模型构建特征，然后将特征经过one-hot编码作为新的特征输入到LR模型训练。
grd_lm.fit(grd_enc.transform(grd.apply(X_train_lr)[:, :, 0]), y_train_lr)
# 用训练好的LR模型多X_test做预测
y_pred_grd_lm = grd_lm.predict_proba(grd_enc.transform(grd.apply(X_test)[:, :, 0]))[:, 1]
# 根据预测结果输出
fpr_grd_lm, tpr_grd_lm, _ = roc_curve(y_test, y_pred_grd_lm)

In [17]:
fpr_grd_lm

array([0.        , 0.00389844, 0.00429828, 0.00479808, 0.00954618,
       0.00964614, 0.01054578, 0.01059576, 0.01089564, 0.01259496,
       0.01304478, 0.01369452, 0.01419432, 0.01439424, 0.0154938 ,
       0.0154938 , 0.01559376, 0.01669332, 0.017493  , 0.01759296,
       0.01784286, 0.02009196, 0.0212415 , 0.02169132, 0.02179128,
       0.02783886, 0.02783886, 0.03043782, 0.03063774, 0.0307377 ,
       0.0307377 , 0.03353659, 0.03753499, 0.03753499, 0.03753499,
       0.05767693, 0.05792683, 0.05802679, 0.06297481, 0.06387445,
       0.06387445, 0.06392443, 0.06817273, 0.07087165, 0.07087165,
       0.08056777, 0.08641543, 0.08666533, 0.08671531, 0.09186325,
       0.09216313, 0.11515394, 0.11520392, 0.11520392, 0.12789884,
       0.12984806, 0.129998  , 0.13059776, 0.1307477 , 0.13429628,
       0.13429628, 0.13434626, 0.13439624, 0.13459616, 0.13469612,
       0.1359956 , 0.13794482, 0.13794482, 0.13869452, 0.1454918 ,
       0.14639144, 0.1482407 , 0.14914034, 0.14944022, 0.18397

In [18]:
tpr_grd_lm

array([0.        , 0.2570028 , 0.2570028 , 0.27786114, 0.53611445,
       0.53701481, 0.56917767, 0.56917767, 0.56917767, 0.6094938 ,
       0.6094938 , 0.62359944, 0.63945578, 0.63945578, 0.67441977,
       0.67592037, 0.67592037, 0.67892157, 0.69422769, 0.69497799,
       0.69582833, 0.72188876, 0.73039216, 0.73564426, 0.73879552,
       0.77370948, 0.77380952, 0.79486795, 0.79826931, 0.79936975,
       0.79946979, 0.81057423, 0.82382953, 0.82397959, 0.82402961,
       0.86154462, 0.86159464, 0.86159464, 0.86809724, 0.86854742,
       0.86864746, 0.86879752, 0.87479992, 0.87715086, 0.87720088,
       0.88990596, 0.89715886, 0.89730892, 0.897509  , 0.90546218,
       0.90571228, 0.92897159, 0.92902161, 0.92907163, 0.94077631,
       0.94342737, 0.94342737, 0.94362745, 0.94362745, 0.94717887,
       0.94732893, 0.94737895, 0.94737895, 0.94742897, 0.94742897,
       0.94822929, 0.94952981, 0.94957983, 0.94987995, 0.95458183,
       0.95493197, 0.95558223, 0.95583233, 0.95593237, 0.97408

In [19]:
_

array([1.99590883e+00, 9.95908828e-01, 9.95109239e-01, 9.80428560e-01,
       9.77762552e-01, 9.77219639e-01, 9.76785022e-01, 9.76674329e-01,
       9.72351590e-01, 9.68545613e-01, 9.62598927e-01, 9.42482660e-01,
       9.38812328e-01, 9.27664170e-01, 9.22909880e-01, 9.15789697e-01,
       9.14143422e-01, 9.09144754e-01, 9.00481257e-01, 8.90607600e-01,
       8.85692489e-01, 8.83719447e-01, 8.81054123e-01, 8.63399388e-01,
       8.52028756e-01, 8.47602821e-01, 8.34752963e-01, 8.22677300e-01,
       8.01610959e-01, 7.81387866e-01, 7.52307031e-01, 7.47457135e-01,
       7.34843002e-01, 7.24189448e-01, 7.00805018e-01, 6.83785042e-01,
       6.58387282e-01, 6.15902369e-01, 6.15486725e-01, 5.83707315e-01,
       5.83151151e-01, 5.79639871e-01, 5.77324689e-01, 5.54430174e-01,
       5.46511652e-01, 5.34482890e-01, 5.32472512e-01, 5.21905683e-01,
       5.15985845e-01, 5.09624647e-01, 5.09162710e-01, 5.00644342e-01,
       4.93164696e-01, 4.77375999e-01, 4.70377401e-01, 4.66146381e-01,
      